### Import the required libraries and modules:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/Blood_Vessel_Results/ALL_BEST_RESULTS/BV_OD_HRF

Mounted at /content/gdrive
/content/gdrive/MyDrive/Blood_Vessel_Results/ALL_BEST_RESULTS/BV_OD_HRF


### Unzip the data:

In [ ]:
# !unzip new_data.zip

### Construct the Model:

In [ ]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(2, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

### Adding the Metrics:

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def DiceBCELoss(y_true, y_pred):

    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(inputs * targets)
    dice_loss = 1 - (2.*intersection + smooth)/(tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets) + dice_loss

    return Dice_BCE

### Training the Model:

In [ ]:
W = 512
H = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (512, 512, 3)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = x[:,:,0:2] # Discard the last channel 

    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 2])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir("files_working")

""" Hyperparameters """
batch_size = 2
lr = 1e-4
num_epochs = 50
model_path = os.path.join("files_working", "model.h5")
csv_path = os.path.join("files_working", "data.csv")

""" Dataset """
dataset_path = "new_data"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

""" Model """
model = build_unet((H, W, 3))
# model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
# model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[iou, dice_coef, dice_coef_OD, dice_coef_MA, Recall(), Precision()])
# model.summary()

model.compile(loss=DiceBCELoss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Train: 40 - 40
Valid: 5 - 5
Epoch 1/50
20/20 [==============================] - ETA: 0s - loss: 1.3693 - dice_coef: 0.1587 - iou: 0.0864 - recall: 0.7413 - precision: 0.1825
Epoch 00001: val_loss improved from inf to 1.58401, saving model to files_working/model.h5
20/20 [==============================] - 81s 2s/step - loss: 1.3693 - dice_coef: 0.1587 - iou: 0.0864 - recall: 0.7413 - precision: 0.1825 - val_loss: 1.5840 - val_dice_coef: 0.0908 - val_iou: 0.0476 - val_recall: 0.0114 - val_precision: 0.3079 - lr: 1.0000e-04
Epoch 2/50
20/20 [==============================] - ETA: 0s - loss: 1.1096 - dice_coef: 0.2238 - iou: 0.1261 - recall: 0.6879 - precision: 0.5071
Epoch 00002: val_loss improved from 1.58401 to 1.53412, saving model to files_working/model.h5
20/20 [==============================] - 39s 2s/step - loss: 1.1096 - dice_coef: 0.2238 - iou: 0.1261 - recall: 0.6879 - precision: 0.5071 - val_loss: 1.5341 - val_dice_coef: 0.0899 - val_iou: 0.0471 - val_recall: 0.0000e+00 - val_p

### Evaluate The Model:

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score, auc
from metrics import dice_loss, dice_coef, iou
from utils import get_filenames_sorted

In [ ]:
H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = x[:,:,0:2] # Discard the last channel 
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path, channel):
    line = np.ones((y_pred.shape[0], 10, 3)) * 255

    pred_image = np.zeros((y_pred.shape[0], y_pred.shape[1], 3))
    _y_pred = y_pred[:, :, channel]
    _ori_y = ori_y[:, :, channel]
    pred_image[:, :, 0] = ((_y_pred > 0.5) & (_ori_y <= 128)) * 255
    pred_image[:, :, 1] = ((_y_pred > 0.5) & (_ori_y  > 128)) * 255
    pred_image[:, :, 2] = ((_ori_y  > 128) & (_y_pred <= 0.5 )) * 255

    print(" saving result", save_image_path)
    cv2.imwrite(save_image_path, pred_image)

In [ ]:
data_dir = "new_data"
od_channel, bv_channel = 0, 1

od_result_dir = "files_demo_od"
bv_result_dir = "files_demo_bv"

create_dir(od_result_dir)
create_dir(bv_result_dir)

model_dir = "files_working"

""" Load the model """
model_file_name = f"{model_dir}/model.h5"
print(model_file_name)
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_coef_OD': dice_coef_OD, 'dice_coef_MA': dice_coef_MA, 'dice_loss': dice_loss}):
model = tf.keras.models.load_model(model_file_name, compile = False)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
# test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

dataset_path = os.path.join("new_data", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE_BV, SCORE_OD = [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_bv = f"{bv_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_bv, bv_channel)


#     """ Calculate the od metrics """
    bv_pred = y_pred[:, :, bv_channel].flatten()
    bv_gt = y[:, :, bv_channel].flatten()
    acc_value = accuracy_score(bv_gt > 0.5, bv_pred>0.5)
    f1_value = f1_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > 0.5) & (bv_pred > 0.5)) / np.sum(bv_gt > 0.5)
    precision_value = precision_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    SCORE_BV.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed])

#     """ Calculate the ma metrics """
    bv_pred = y_pred[:, :, od_channel].flatten()
    bv_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(bv_gt > 0.5, bv_pred>0.5)
    f1_value = f1_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > 0.5) & (bv_pred > 0.5)) / np.sum(bv_gt > 0.5)
    precision_value = precision_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed])
    
print("\n")
for SCORE in [SCORE_OD, SCORE_BV]:
    if SCORE == SCORE_OD:
        print("****** OD ******Metrics")
    else:
        print("****** BV ******Metrics")
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f} (dice score)")
    print(f"AUC: {score[1]:0.5f} (Auc score)")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed"])
        df.to_csv(f"{od_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed"])
        df.to_csv(f"{bv_result_dir}/score.csv")
    print("\n")

files_working/model.h5


  0%|          | 0/5 [00:00<?, ?it/s]

11_h
255 1.0
 saving result files_demo_od/11_h.png
 saving result files_demo_bv/11_h.png


 20%|██        | 1/5 [00:01<00:05,  1.49s/it]

12_h
255 1.0
 saving result files_demo_od/12_h.png
 saving result files_demo_bv/12_h.png


...

14_h
255 1.0
 saving result files_demo_od/14_h.png
 saving result files_demo_bv/14_h.png


 80%|████████  | 4/5 [00:04<00:01,  1.02s/it]

15_h
255 1.0
 saving result files_demo_od/15_h.png
 saving result files_demo_bv/15_h.png


100%|██████████| 5/5 [00:05<00:00,  1.05s/it]



****** OD ******Metrics
Accuracy: 0.99673
F1: 0.83973 (dice score)
AUC: 0.83973 (Auc score)
Jaccard: 0.72962
Recall: 0.74850
Precision: 0.97154


****** BV ******Metrics
Accuracy: 0.96837
F1: 0.81661 (dice score)
AUC: 0.81661 (Auc score)
Jaccard: 0.69074
Recall: 0.79067
Precision: 0.84549




In [ ]:
data_dir = "new_data"
od_channel, bv_channel = 0, 1

od_result_dir = "files_demo_od_75"
bv_result_dir = "files_demo_bv_75"

create_dir(od_result_dir)
create_dir(bv_result_dir)

model_dir = "files_working"

""" Load the model """
model_file_name = f"{model_dir}/model.h5"
print(model_file_name)
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_coef_OD': dice_coef_OD, 'dice_coef_MA': dice_coef_MA, 'dice_loss': dice_loss}):
model = tf.keras.models.load_model(model_file_name, compile = False)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
# test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

dataset_path = os.path.join("new_data", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE_BV, SCORE_OD = [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_bv = f"{bv_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_bv, bv_channel)


#     """ Calculate the od metrics """
    bv_pred = y_pred[:, :, bv_channel].flatten()
    bv_gt = y[:, :, bv_channel].flatten()
    acc_value = accuracy_score(bv_gt > 0.75, bv_pred>75)
    f1_value = f1_score(bv_gt > 0.75, bv_pred>0.75, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > 0.75, bv_pred>0.75, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > 0.75, bv_pred>0.75, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > 0.75) & (bv_pred > 0.75)) / np.sum(bv_gt > 0.75)
    precision_value = precision_score(bv_gt > 0.75, bv_pred>0.75, labels=[0, 1], average="binary")
    SCORE_BV.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed])

#     """ Calculate the ma metrics """
    bv_pred = y_pred[:, :, od_channel].flatten()
    bv_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(bv_gt > 0.75, bv_pred>0.75)
    f1_value = f1_score(bv_gt > 0.75, bv_pred>0.75, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > 0.75, bv_pred>0.75, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > 0.75, bv_pred>0.75, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > 0.75) & (bv_pred > 0.75)) / np.sum(bv_gt > 0.75)
    precision_value = precision_score(bv_gt > 0.75, bv_pred>0.75, labels=[0, 1], average="binary")
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed])
    
print("\n")
for SCORE in [SCORE_OD, SCORE_BV]:
    if SCORE == SCORE_OD:
        print("****** OD ******Metrics")
    else:
        print("****** BV ******Metrics")
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f} (dice score)")
    print(f"AUC: {score[1]:0.5f} (Auc score)")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed"])
        df.to_csv(f"{od_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed"])
        df.to_csv(f"{bv_result_dir}/score.csv")
    print("\n")

files_working/model.h5


  0%|          | 0/5 [00:00<?, ?it/s]

11_h
255 1.0
 saving result files_demo_od_75/11_h.png
 saving result files_demo_bv_75/11_h.png


 20%|██        | 1/5 [00:23<01:33, 23.31s/it]

12_h
255 1.0
 saving result files_demo_od_75/12_h.png
 saving result files_demo_bv_75/12_h.png


...

14_h
255 1.0
 saving result files_demo_od_75/14_h.png
 saving result files_demo_bv_75/14_h.png


 80%|████████  | 4/5 [00:29<00:04,  4.99s/it]

15_h
255 1.0
 saving result files_demo_od_75/15_h.png
 saving result files_demo_bv_75/15_h.png


100%|██████████| 5/5 [00:31<00:00,  6.28s/it]



****** OD ******Metrics
Accuracy: 0.99675
F1: 0.84020 (dice score)
AUC: 0.84020 (Auc score)
Jaccard: 0.73027
Recall: 0.74987
Precision: 0.97058


****** BV ******Metrics
Accuracy: 0.91419
F1: 0.81191 (dice score)
AUC: 0.81191 (Auc score)
Jaccard: 0.68403
Recall: 0.80545
Precision: 0.81951




In [ ]:
data_dir = "new_data"
od_channel, bv_channel = 0, 1

od_result_dir = "files_demo_od_75"
bv_result_dir = "files_demo_bv_75"

create_dir(od_result_dir)
create_dir(bv_result_dir)

model_dir = "files_working"
threshold = 0.9

""" Load the model """
model_file_name = f"{model_dir}/model.h5"
print(model_file_name)
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_coef_OD': dice_coef_OD, 'dice_coef_MA': dice_coef_MA, 'dice_loss': dice_loss}):
model = tf.keras.models.load_model(model_file_name, compile = False)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
# test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

dataset_path = os.path.join("new_data", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE_BV, SCORE_OD = [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred_prob = y_pred
    y_pred = y_pred > threshold
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_bv = f"{bv_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_bv, bv_channel)


#     """ Calculate the od metrics """
    bv_pred = y_pred[:, :, bv_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, bv_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_BV.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])

#     """ Calculate the ma metrics """
    bv_pred = y_pred[:, :, od_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])
    
print("\n")
for SCORE in [SCORE_OD, SCORE_BV]:
    if SCORE == SCORE_OD:
        print("****** OD ******Metrics")
    else:
        print("****** BV ******Metrics")
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Threshold:", threshold)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f} (dice score)")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")
    print(f"AUC: {score[5]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{od_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{bv_result_dir}/score.csv")
    print("\n")

files_working/model.h5


  0%|          | 0/5 [00:00<?, ?it/s]

11_h
255 1.0
 saving result files_demo_od_75/11_h.png
 saving result files_demo_bv_75/11_h.png


 20%|██        | 1/5 [00:02<00:10,  2.54s/it]

12_h
255 1.0
 saving result files_demo_od_75/12_h.png
 saving result files_demo_bv_75/12_h.png


...

14_h
255 1.0
 saving result files_demo_od_75/14_h.png
 saving result files_demo_bv_75/14_h.png


 80%|████████  | 4/5 [00:07<00:01,  1.61s/it]

15_h
255 1.0
 saving result files_demo_od_75/15_h.png
 saving result files_demo_bv_75/15_h.png


100%|██████████| 5/5 [00:08<00:00,  1.71s/it]



****** OD ******Metrics
Threshold: 0.9
Accuracy: 0.99619
F1: 0.80448 (dice score)
Jaccard: 0.68015
Recall: 0.68731
Precision: 0.98649
AUC: 0.68731


****** BV ******Metrics
Threshold: 0.9
Accuracy: 0.96874
F1: 0.78609 (dice score)
Jaccard: 0.64896
Recall: 0.70682
Precision: 0.88789
AUC: 0.70682




In [ ]:
from copy import deepcopy

data_dir = "new_data"
od_channel, bv_channel = 0, 1

od_result_dir = "files_demo_od_75"
bv_result_dir = "files_demo_bv_75"

create_dir(od_result_dir)
create_dir(bv_result_dir)

model_dir = "files_working"
threshold = 0.75

""" Load the model """
model_file_name = f"{model_dir}/model.h5"
print(model_file_name)
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_coef_OD': dice_coef_OD, 'dice_coef_MA': dice_coef_MA, 'dice_loss': dice_loss}):
model = tf.keras.models.load_model(model_file_name, compile = False)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
# test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

dataset_path = os.path.join("new_data", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE_BV, SCORE_OD = [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred_prob = deepcopy(y_pred)
    y_pred = y_pred > threshold
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_bv = f"{bv_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_bv, bv_channel)


#     """ Calculate the od metrics """
    bv_pred = y_pred[:, :, bv_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, bv_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_BV.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])

#     """ Calculate the ma metrics """
    bv_pred = y_pred[:, :, od_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])
    
print("\n")
for SCORE in [SCORE_OD, SCORE_BV]:
    if SCORE == SCORE_OD:
        print("****** OD ******Metrics")
    else:
        print("****** BV ******Metrics")
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Threshold:", threshold)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f} (dice score)")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")
    print(f"AUC: {score[5]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{od_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{bv_result_dir}/score.csv")
    print("\n")

files_working/model.h5


  0%|          | 0/5 [00:00<?, ?it/s]

11_h
255 1.0
 saving result files_demo_od_75/11_h.png
 saving result files_demo_bv_75/11_h.png


 20%|██        | 1/5 [00:01<00:06,  1.51s/it]

12_h
255 1.0
 saving result files_demo_od_75/12_h.png
 saving result files_demo_bv_75/12_h.png


...

14_h
255 1.0
 saving result files_demo_od_75/14_h.png
 saving result files_demo_bv_75/14_h.png


 80%|████████  | 4/5 [00:04<00:01,  1.06s/it]

15_h
255 1.0
 saving result files_demo_od_75/15_h.png
 saving result files_demo_bv_75/15_h.png


100%|██████████| 5/5 [00:05<00:00,  1.10s/it]



****** OD ******Metrics
Threshold: 0.75
Accuracy: 0.99646
F1: 0.82166 (dice score)
Jaccard: 0.70423
Recall: 0.71624
Precision: 0.97982
AUC: 0.71624


****** BV ******Metrics
Threshold: 0.75
Accuracy: 0.96904
F1: 0.80421 (dice score)
Jaccard: 0.67351
Recall: 0.75289
Precision: 0.86470
AUC: 0.75289




In [ ]:
from copy import deepcopy

data_dir = "new_data"
od_channel, bv_channel = 0, 1

od_result_dir = "files_demo_od_75"
bv_result_dir = "files_demo_bv_75"

create_dir(od_result_dir)
create_dir(bv_result_dir)

model_dir = "files_working"
threshold = 0.5

""" Load the model """
model_file_name = f"{model_dir}/model.h5"
print(model_file_name)
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_coef_OD': dice_coef_OD, 'dice_coef_MA': dice_coef_MA, 'dice_loss': dice_loss}):
model = tf.keras.models.load_model(model_file_name, compile = False)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
# test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

dataset_path = os.path.join("new_data", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE_BV, SCORE_OD = [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred_prob = deepcopy(y_pred)
    y_pred = y_pred > threshold
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_bv = f"{bv_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_bv, bv_channel)


#     """ Calculate the od metrics """
    bv_pred = y_pred[:, :, bv_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, bv_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_BV.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])

#     """ Calculate the ma metrics """
    bv_pred = y_pred[:, :, od_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])
    
print("\n")
for SCORE in [SCORE_OD, SCORE_BV]:
    if SCORE == SCORE_OD:
        print("****** OD ******Metrics")
    else:
        print("****** BV ******Metrics")
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Threshold:", threshold)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f} (dice score)")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")
    print(f"AUC: {score[5]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{od_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{bv_result_dir}/score.csv")
    print("\n")

files_working/model.h5


  0%|          | 0/5 [00:00<?, ?it/s]

11_h
255 1.0
 saving result files_demo_od_75/11_h.png
 saving result files_demo_bv_75/11_h.png


 20%|██        | 1/5 [00:02<00:09,  2.28s/it]

12_h
255 1.0
 saving result files_demo_od_75/12_h.png
 saving result files_demo_bv_75/12_h.png


...

14_h
255 1.0
 saving result files_demo_od_75/14_h.png
 saving result files_demo_bv_75/14_h.png


 80%|████████  | 4/5 [00:06<00:01,  1.63s/it]

15_h
255 1.0
 saving result files_demo_od_75/15_h.png
 saving result files_demo_bv_75/15_h.png


100%|██████████| 5/5 [00:08<00:00,  1.61s/it]



****** OD ******Metrics
Threshold: 0.5
Accuracy: 0.99673
F1: 0.83973 (dice score)
Jaccard: 0.72962
Recall: 0.74850
Precision: 0.97154
AUC: 0.74850


****** BV ******Metrics
Threshold: 0.5
Accuracy: 0.96837
F1: 0.81661 (dice score)
Jaccard: 0.69074
Recall: 0.79067
Precision: 0.84549
AUC: 0.79067




In [ ]:
from copy import deepcopy

data_dir = "new_data"
od_channel, bv_channel = 0, 1

od_result_dir = "files_demo_od_75"
bv_result_dir = "files_demo_bv_75"

create_dir(od_result_dir)
create_dir(bv_result_dir)

model_dir = "files_working"
threshold = 0.3

""" Load the model """
model_file_name = f"{model_dir}/model.h5"
print(model_file_name)
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_coef_OD': dice_coef_OD, 'dice_coef_MA': dice_coef_MA, 'dice_loss': dice_loss}):
model = tf.keras.models.load_model(model_file_name, compile = False)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
# test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

dataset_path = os.path.join("new_data", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE_BV, SCORE_OD = [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred_prob = deepcopy(y_pred)
    y_pred = y_pred > threshold
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_bv = f"{bv_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_bv, bv_channel)


#     """ Calculate the od metrics """
    bv_pred = y_pred[:, :, bv_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, bv_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_BV.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])

#     """ Calculate the ma metrics """
    bv_pred = y_pred[:, :, od_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])
    
print("\n")
for SCORE in [SCORE_OD, SCORE_BV]:
    if SCORE == SCORE_OD:
        print("****** OD ******Metrics")
    else:
        print("****** BV ******Metrics")
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Threshold:", threshold)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f} (dice score)")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")
    print(f"AUC: {score[5]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{od_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{bv_result_dir}/score.csv")
    print("\n")

files_working/model.h5


  0%|          | 0/5 [00:00<?, ?it/s]

11_h
255 1.0
 saving result files_demo_od_75/11_h.png
 saving result files_demo_bv_75/11_h.png


 20%|██        | 1/5 [00:02<00:08,  2.23s/it]

12_h
255 1.0
 saving result files_demo_od_75/12_h.png
 saving result files_demo_bv_75/12_h.png


...

14_h
255 1.0
 saving result files_demo_od_75/14_h.png
 saving result files_demo_bv_75/14_h.png


 80%|████████  | 4/5 [00:07<00:01,  1.74s/it]

15_h
255 1.0
 saving result files_demo_od_75/15_h.png
 saving result files_demo_bv_75/15_h.png


100%|██████████| 5/5 [00:08<00:00,  1.68s/it]



****** OD ******Metrics
Threshold: 0.3
Accuracy: 0.99692
F1: 0.85191 (dice score)
Jaccard: 0.74688
Recall: 0.77206
Precision: 0.96427
AUC: 0.77206


****** BV ******Metrics
Threshold: 0.3
Accuracy: 0.96702
F1: 0.82213 (dice score)
Jaccard: 0.69846
Recall: 0.82216
Precision: 0.82297
AUC: 0.82216




In [ ]:
from copy import deepcopy

data_dir = "new_data"
od_channel, bv_channel = 0, 1

od_result_dir = "files_demo_od_75"
bv_result_dir = "files_demo_bv_75"

create_dir(od_result_dir)
create_dir(bv_result_dir)

model_dir = "files_working"
threshold = 0.2

""" Load the model """
model_file_name = f"{model_dir}/model.h5"
print(model_file_name)
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_coef_OD': dice_coef_OD, 'dice_coef_MA': dice_coef_MA, 'dice_loss': dice_loss}):
model = tf.keras.models.load_model(model_file_name, compile = False)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
# test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

dataset_path = os.path.join("new_data", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE_BV, SCORE_OD = [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred_prob = deepcopy(y_pred)
    y_pred = y_pred > threshold
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_bv = f"{bv_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_bv, bv_channel)


#     """ Calculate the od metrics """
    bv_pred = y_pred[:, :, bv_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, bv_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_BV.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])

#     """ Calculate the ma metrics """
    bv_pred = y_pred[:, :, od_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])
    
print("\n")
for SCORE in [SCORE_OD, SCORE_BV]:
    if SCORE == SCORE_OD:
        print("****** OD ******Metrics")
    else:
        print("****** BV ******Metrics")
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Threshold:", threshold)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f} (dice score)")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")
    print(f"AUC: {score[5]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{od_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{bv_result_dir}/score.csv")
    print("\n")

files_working/model.h5


  0%|          | 0/5 [00:00<?, ?it/s]

11_h
255 1.0
 saving result files_demo_od_75/11_h.png
 saving result files_demo_bv_75/11_h.png


 20%|██        | 1/5 [00:22<01:29, 22.42s/it]

12_h
255 1.0
 saving result files_demo_od_75/12_h.png
 saving result files_demo_bv_75/12_h.png


...

14_h
255 1.0
 saving result files_demo_od_75/14_h.png
 saving result files_demo_bv_75/14_h.png


 80%|████████  | 4/5 [00:27<00:04,  4.54s/it]

15_h
255 1.0
 saving result files_demo_od_75/15_h.png
 saving result files_demo_bv_75/15_h.png


100%|██████████| 5/5 [00:29<00:00,  5.86s/it]




****** OD ******Metrics
Threshold: 0.2
Accuracy: 0.99701
F1: 0.85878 (dice score)
Jaccard: 0.75635
Recall: 0.78752
Precision: 0.95769
AUC: 0.78752


****** BV ******Metrics
Threshold: 0.2
Accuracy: 0.96516
F1: 0.82098 (dice score)
Jaccard: 0.69669
Recall: 0.84221
Precision: 0.80153
AUC: 0.84221




In [ ]:
from copy import deepcopy

data_dir = "new_data"
od_channel, bv_channel = 0, 1

od_result_dir = "files_demo_od_75"
bv_result_dir = "files_demo_bv_75"

create_dir(od_result_dir)
create_dir(bv_result_dir)

model_dir = "files_working"
threshold = 0.25

""" Load the model """
model_file_name = f"{model_dir}/model.h5"
print(model_file_name)
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_coef_OD': dice_coef_OD, 'dice_coef_MA': dice_coef_MA, 'dice_loss': dice_loss}):
model = tf.keras.models.load_model(model_file_name, compile = False)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
# test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

dataset_path = os.path.join("new_data", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE_BV, SCORE_OD = [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred_prob = deepcopy(y_pred)
    y_pred = y_pred > threshold
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_bv = f"{bv_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_bv, bv_channel)


#     """ Calculate the od metrics """
    bv_pred = y_pred[:, :, bv_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, bv_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_BV.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])

#     """ Calculate the ma metrics """
    bv_pred = y_pred[:, :, od_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])
    
print("\n")
for SCORE in [SCORE_OD, SCORE_BV]:
    if SCORE == SCORE_OD:
        print("****** OD ******Metrics")
    else:
        print("****** BV ******Metrics")
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Threshold:", threshold)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f} (dice score)")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")
    print(f"AUC: {score[5]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{od_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{bv_result_dir}/score.csv")
    print("\n")

files_working/model.h5


  0%|          | 0/5 [00:00<?, ?it/s]

11_h
255 1.0
 saving result files_demo_od_75/11_h.png
 saving result files_demo_bv_75/11_h.png


 20%|██        | 1/5 [00:04<00:17,  4.39s/it]

12_h
255 1.0
 saving result files_demo_od_75/12_h.png
 saving result files_demo_bv_75/12_h.png


...

14_h
255 1.0
 saving result files_demo_od_75/14_h.png
 saving result files_demo_bv_75/14_h.png


 80%|████████  | 4/5 [00:13<00:03,  3.19s/it]

15_h
255 1.0
 saving result files_demo_od_75/15_h.png
 saving result files_demo_bv_75/15_h.png


100%|██████████| 5/5 [00:16<00:00,  3.27s/it]



****** OD ******Metrics
Threshold: 0.25
Accuracy: 0.99696
F1: 0.85507 (dice score)
Jaccard: 0.75131
Recall: 0.77907
Precision: 0.96148
AUC: 0.77907


****** BV ******Metrics
Threshold: 0.25
Accuracy: 0.96630
F1: 0.82231 (dice score)
Jaccard: 0.69867
Recall: 0.83153
Precision: 0.81414
AUC: 0.83153




In [ ]:
from copy import deepcopy

data_dir = "new_data"
od_channel, bv_channel = 0, 1

od_result_dir = "files_demo_od_75"
bv_result_dir = "files_demo_bv_75"

create_dir(od_result_dir)
create_dir(bv_result_dir)

model_dir = "files_working"
threshold = 0.35

""" Load the model """
model_file_name = f"{model_dir}/model.h5"
print(model_file_name)
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_coef_OD': dice_coef_OD, 'dice_coef_MA': dice_coef_MA, 'dice_loss': dice_loss}):
model = tf.keras.models.load_model(model_file_name, compile = False)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
# test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

dataset_path = os.path.join("new_data", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE_BV, SCORE_OD = [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred_prob = deepcopy(y_pred)
    y_pred = y_pred > threshold
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_bv = f"{bv_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_bv, bv_channel)


#     """ Calculate the od metrics """
    bv_pred = y_pred[:, :, bv_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, bv_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_BV.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])

#     """ Calculate the ma metrics """
    bv_pred = y_pred[:, :, od_channel].flatten()
    bv_pred_prob = y_pred_prob[:, :, bv_channel].flatten()
    bv_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(bv_gt > threshold, bv_pred>threshold)
    f1_value = f1_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > threshold) & (bv_pred > threshold)) / np.sum(bv_gt > threshold)
    precision_value = precision_score(bv_gt > threshold, bv_pred>threshold, labels=[0, 1], average="binary")
    auc_score = auc(bv_gt > threshold, bv_pred_prob)
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed, auc_score])
    
print("\n")
for SCORE in [SCORE_OD, SCORE_BV]:
    if SCORE == SCORE_OD:
        print("****** OD ******Metrics")
    else:
        print("****** BV ******Metrics")
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Threshold:", threshold)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f} (dice score)")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")
    print(f"AUC: {score[5]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{od_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed", "AUC"])
        df.to_csv(f"{bv_result_dir}/score.csv")
    print("\n")

files_working/model.h5


  0%|          | 0/5 [00:00<?, ?it/s]

11_h
255 1.0
 saving result files_demo_od_75/11_h.png
 saving result files_demo_bv_75/11_h.png


 20%|██        | 1/5 [00:04<00:18,  4.53s/it]

12_h
255 1.0
 saving result files_demo_od_75/12_h.png
 saving result files_demo_bv_75/12_h.png


...

14_h
255 1.0
 saving result files_demo_od_75/14_h.png
 saving result files_demo_bv_75/14_h.png


 80%|████████  | 4/5 [00:12<00:02,  2.89s/it]

15_h
255 1.0
 saving result files_demo_od_75/15_h.png
 saving result files_demo_bv_75/15_h.png


100%|██████████| 5/5 [00:15<00:00,  3.02s/it]



****** OD ******Metrics
Threshold: 0.35
Accuracy: 0.99688
F1: 0.84902 (dice score)
Jaccard: 0.74285
Recall: 0.76607
Precision: 0.96670
AUC: 0.76607


****** BV ******Metrics
Threshold: 0.35
Accuracy: 0.96752
F1: 0.82134 (dice score)
Jaccard: 0.69736
Recall: 0.81358
Precision: 0.83015
AUC: 0.81358


